In [ ]:
import imaplib
import email
from email.header import decode_header

def decode_mime_words(s):
    decoded_words = decode_header(s)
    decoded_string = ""
    for word, encoding in decoded_words:
        if isinstance(word, bytes):
            encoding = encoding or "utf-8"
            try:
                word = word.decode(encoding)
            except (UnicodeDecodeError, LookupError):
                word = word.decode("utf-8", errors="ignore")
        decoded_string += word
    return decoded_string

def fetch_unread_emails_from_sender(email_user, email_password, imap_server, sender_email):
    unread_emails = []
    mail = imaplib.IMAP4_SSL(imap_server)
    mail.login(email_user, email_password)
    mail.select("inbox")

    # Search for unread emails from the specific sender
    search_criterion = f'(UNSEEN FROM "{sender_email}")'
    result, data = mail.search(None, search_criterion)
    if result == "OK":
        for num in data[0].split()[:10]:  # Limit to first 10 unread emails
            result, data = mail.fetch(num, "(RFC822)")
            if result == "OK":
                email_message = email.message_from_bytes(data[0][1])
                email_subject = decode_mime_words(email_message["Subject"])

                email_from = decode_mime_words(email_message.get("From"))
                email_body = ""
                
                def get_body(part):
                    content_type = part.get_content_type()
                    if "plain" in content_type.lower():
                        try:
                            return part.get_payload(decode=True).decode()
                        except UnicodeDecodeError:
                            return part.get_payload(decode=True).decode("utf-8", errors="ignore")
                    return ""

                if email_message.is_multipart():
                    for part in email_message.walk():
                        email_body += get_body(part)
                else:
                    email_body = get_body(email_message)
                
                try:
                    unread_emails.append({
                        'subject': email_subject,
                        'from': email_from,
                        'body': email_body
                    })
                except UnicodeDecodeError:
                    print(f"Skipping email due to decoding error.")
    
    mail.close()
    mail.logout()
    return unread_emails


In [ ]:
!pip install fastapi
!pip install pydantic
!pip install smtplib
!pip install imapclient
!pip install email
!pip install python-multipart
!pip install logging


In [ ]:
from fastapi import FastAPI, BackgroundTasks, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import smtplib
import imaplib
import email
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import logging
from datetime import datetime
import json
from pathlib import Path

In [ ]:
from fastapi import FastAPI, BackgroundTasks, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import smtplib
import imaplib
import email
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import logging
from datetime import datetime
import json
from pathlib import Path
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load configuration from environment variables
GMAIL_USER = os.getenv("xtractpay@gmail.com")
GMAIL_PASSWORD = os.getenv("xtractpay@1234")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

app = FastAPI()

class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        
    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        """Generate HTML email content with expense reports and action buttons."""
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        
        for report in expense_reports:
            approve_url = f"{API_BASE_URL}/approve/{report.id}"
            reject_url = f"{API_BASE_URL}/reject/{report.id}"
            
            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """
            
        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        """Send summary email to manager with expense reports."""
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            # Create HTML content
            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            # Attach receipts if available
            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                        filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            # Send email
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        """Process incoming email replies for approvals/rejections."""
        try:
            # Connect to IMAP server
            mail = imaplib.IMAP4_SSL(self.imap_server)
            mail.login(GMAIL_USER, GMAIL_PASSWORD)
            mail.select('inbox')

            # Search for unread emails
            _, messages = mail.search(None, 'UNSEEN')
            
            for msg_num in messages[0].split():
                _, msg_data = mail.fetch(msg_num, '(RFC822)')
                email_body = msg_data[0][1]
                email_message = email.message_from_bytes(email_body)
                
                # Process email content
                subject = email_message['subject']
                if 'Re: Expense Reports Requiring Approval' in subject:
                    content = self._get_email_content(email_message)
                    
                    # Extract expense report ID and decision
                    report_id = self._extract_report_id(content)
                    decision = self._extract_decision(content)
                    
                    if report_id and decision:
                        self._log_decision(report_id, decision)
                        logger.info(f"Processed decision: {decision} for report {report_id}")

            return {"status": "success", "message": "Processed email replies"}

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    def _get_email_content(self, email_message) -> str:
        """Extract email content from message."""
        if email_message.is_multipart():
            for part in email_message.walk():
                if part.get_content_type() == "text/plain":
                    return part.get_payload(decode=True).decode()
        return email_message.get_payload(decode=True).decode()

    def _extract_report_id(self, content: str) -> Optional[str]:
        """Extract expense report ID from email content."""
        # Implementation would depend on email format
        # This is a simple example
        if "ID:" in content:
            return content.split("ID:")[1].split()[0]
        return None

    def _extract_decision(self, content: str) -> Optional[str]:
        """Extract approval/rejection decision from email content."""
        content = content.lower()
        if "approve" in content:
            return "approved"
        elif "reject" in content:
            return "rejected"
        return None

    def _log_decision(self, report_id: str, decision: str):
        """Log manager's decision."""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        
        # Save to log file
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

email_service = EmailService()


@app.post("/send-summary-email")
async def send_summary_email(request: EmailRequest, background_tasks: BackgroundTasks):
    """API endpoint to send summary email."""
    background_tasks.add_task(email_service.send_summary_email, request)
    return {"status": "success", "message": "Email sending initiated"}

@app.post("/process-email-replies")
async def process_email_replies(background_tasks: BackgroundTasks):
    """API endpoint to process manager replies."""
    background_tasks.add_task(email_service.process_email_replies)
    return {"status": "success", "message": "Email processing initiated"}

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    """Handle approval via email link click."""
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    """Handle rejection via email link click."""
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

if __name__ == "__main__":
    # Run the app using uvicorn. Make sure to run this file as a script.
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)

In [ ]:
!pip install fastapi uvicorn pydantic nest_asyncio


In [ ]:
import nest_asyncio
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def home():
    return {"message": "FastAPI is running inside Jupyter Notebook!"}

# Apply nest_asyncio to allow event loop usage
nest_asyncio.apply()

# Run the server
import uvicorn
uvicorn.run(app, host="127.0.0.1", port=8000)


In [ ]:
import os
import smtplib
import imaplib
import email
import json
import logging
from datetime import datetime
from pathlib import Path
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel
from typing import List, Optional
import nest_asyncio
import uvicorn

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load configuration from environment variables
GMAIL_USER = os.getenv("GMAIL_USER", "xtractpay@gmail.com")  # Set your email
GMAIL_PASSWORD = os.getenv("GMAIL_PASSWORD", "qzbqhlvmvjsdtana")  # Set your password
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")


class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None


class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]


app = FastAPI()


class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"

    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        """Generate HTML email content with expense reports and action buttons."""
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """

        for report in expense_reports:
            approve_url = f"{API_BASE_URL}/approve/{report.id}"
            reject_url = f"{API_BASE_URL}/reject/{report.id}"

            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """

        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        """Send summary email to manager with expense reports."""
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            # Create HTML content
            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            # Attach receipts if available
            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment',
                                           filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            # Send email
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))


email_service = EmailService()


@app.post("/send-summary-email")
async def send_summary_email(request: EmailRequest, background_tasks: BackgroundTasks):
    """API endpoint to send summary email."""
    background_tasks.add_task(email_service.send_summary_email, request)
    return {"status": "success", "message": "Email sending initiated"}


@app.get("/")
async def home():
    """Simple API test route."""
    return {"message": "FastAPI is running inside Jupyter Notebook!"}


# Apply nest_asyncio to allow event loop usage
nest_asyncio.apply()

# Run the FastAPI server
uvicorn.run(app, host="127.0.0.1", port=8000)


In [ ]:
import os
from fastapi import FastAPI, BackgroundTasks, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import smtplib
import imaplib
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import logging
from datetime import datetime
import json
from pathlib import Path
import uvicorn
from fastapi.middleware.cors import CORSMiddleware

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
class Settings:
    GMAIL_USER = "xtractpay@gmail.com"  # Replace with your email
    GMAIL_PASSWORD = "qzbqhlvmvjsdtana"    # Replace with your app password
    API_BASE_URL = "http://localhost:8000"
    SMTP_SERVER = "smtp.gmail.com"
    SMTP_PORT = 587
    IMAP_SERVER = "imap.gmail.com"
    
    # Ensure logs directory exists
    LOG_DIR = Path("logs")
    LOG_DIR.mkdir(exist_ok=True)
    DECISIONS_LOG = LOG_DIR / "decisions.log"

settings = Settings()

# Data Models
class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

# Email Service
class EmailService:
    def __init__(self):
        self.smtp_server = settings.SMTP_SERVER
        self.smtp_port = settings.SMTP_PORT
        self.imap_server = settings.IMAP_SERVER
        
    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        
        for report in expense_reports:
            approve_url = f"{settings.API_BASE_URL}/approve/{report.id}"
            reject_url = f"{settings.API_BASE_URL}/reject/{report.id}"
            
            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """
            
        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        try:
            msg = MIMEMultipart()
            msg['From'] = settings.GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                        filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(settings.GMAIL_USER, settings.GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        try:
            mail = imaplib.IMAP4_SSL(self.imap_server)
            mail.login(settings.GMAIL_USER, settings.GMAIL_PASSWORD)
            mail.select('inbox')

            _, messages = mail.search(None, 'UNSEEN')
            
            for msg_num in messages[0].split():
                _, msg_data = mail.fetch(msg_num, '(RFC822)')
                email_body = msg_data[0][1]
                email_message = email.message_from_bytes(email_body)
                
                subject = email_message['subject']
                if 'Re: Expense Reports Requiring Approval' in subject:
                    content = self._get_email_content(email_message)
                    report_id = self._extract_report_id(content)
                    decision = self._extract_decision(content)
                    
                    if report_id and decision:
                        self._log_decision(report_id, decision)
                        logger.info(f"Processed decision: {decision} for report {report_id}")

            return {"status": "success", "message": "Processed email replies"}

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    def _get_email_content(self, email_message) -> str:
        if email_message.is_multipart():
            for part in email_message.walk():
                if part.get_content_type() == "text/plain":
                    return part.get_payload(decode=True).decode()
        return email_message.get_payload(decode=True).decode()

    def _extract_report_id(self, content: str) -> Optional[str]:
        if "ID:" in content:
            return content.split("ID:")[1].split()[0]
        return None

    def _extract_decision(self, content: str) -> Optional[str]:
        content = content.lower()
        if "approve" in content:
            return "approved"
        elif "reject" in content:
            return "rejected"
        return None

    def _log_decision(self, report_id: str, decision: str):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        
        with open(settings.DECISIONS_LOG, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

# Create FastAPI app
app = FastAPI(title="Expense Approval System")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Initialize email service
email_service = EmailService()


@app.post("/send-summary-email")
async def send_summary_email(request: EmailRequest, background_tasks: BackgroundTasks):
    """API endpoint to send summary email."""
    background_tasks.add_task(email_service.send_summary_email, request)
    return {"status": "success", "message": "Email sending initiated"}

@app.post("/process-email-replies")
async def process_email_replies(background_tasks: BackgroundTasks):
    """API endpoint to process manager replies."""
    background_tasks.add_task(email_service.process_email_replies)
    return {"status": "success", "message": "Email processing initiated"}

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    """Handle approval via email link click."""
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    """Handle rejection via email link click."""
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

if __name__ == "__main__":
    # Run the app using uvicorn. Make sure to run this file as a script.
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)

In [ ]:
# %% [code]
import os
import json
import logging
import smtplib
import imaplib
import email
from datetime import datetime
from pathlib import Path
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from typing import List, Optional

from fastapi import FastAPI, BackgroundTasks, HTTPException
from pydantic import BaseModel

# For running FastAPI in a Jupyter Notebook
import nest_asyncio
import uvicorn
import threading

# Apply nest_asyncio to allow the event loop to be re-entered in the notebook
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load configuration from environment variables
# (Set these environment variables in your notebook or OS if needed)
GMAIL_USER = os.getenv("GMAIL_USER", "xtractpay@gmail.com")
GMAIL_PASSWORD = os.getenv("GMAIL_APP_PASSWORD", "qzbqhlvmvjsdtana")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

# Data Models
class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

# Initialize FastAPI application
app = FastAPI()

# Email Service Implementation
class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        
    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        """Generate HTML email content with expense reports and action buttons."""
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        for report in expense_reports:
            approve_url = f"{API_BASE_URL}/approve/{report.id}"
            reject_url = f"{API_BASE_URL}/reject/{report.id}"
            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """
        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        """Send summary email to manager with expense reports."""
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            # Create HTML content
            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            # Attach receipts if available
            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                           filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            # Send email via Gmail SMTP
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        """Process incoming email replies and return them in JSON format."""
        try:
            # Connect to IMAP server
            mail = imaplib.IMAP4_SSL(self.imap_server)
            mail.login(GMAIL_USER, GMAIL_PASSWORD)
            mail.select('inbox')

            # Search for unread messages
            _, messages = mail.search(None, 'UNSEEN')
            
            if not messages[0]:
                return {"status": "success", "message": "No new emails found", "replies": []}
                
            replies = []
            for msg_num in messages[0].split():
                try:
                    _, msg_data = mail.fetch(msg_num, '(RFC822)')
                    if not msg_data or not msg_data[0]:
                        continue
                        
                    email_body = msg_data[0][1]
                    email_message = email.message_from_bytes(email_body)
                    
                    # Extract content
                    content = self._get_email_content(email_message)
                    
                    # Create JSON response for each email
                    reply = {
                        "from": email_message['from'],
                        "subject": email_message['subject'],
                        "content": content,
                        "date": email_message['date']
                    }
                    
                    replies.append(reply)
                    
                    # Print JSON output for each reply
                    print(json.dumps(reply, indent=2))
                    
                except Exception as e:
                    logger.error(f"Error processing individual email: {str(e)}")
                    continue
                    
            mail.close()
            mail.logout()
            
            return {"status": "success", "message": f"Processed {len(replies)} emails", "replies": replies}

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))
    
    def _get_email_content(self, email_message) -> str:
        """Extract email content from message, handling both plain text and HTML."""
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain":
                    return part.get_payload(decode=True).decode()
                elif content_type == "text/html":
                    # If no plain text is found, use HTML content
                    return part.get_payload(decode=True).decode()
        else:
            # Handle non-multipart messages
            return email_message.get_payload(decode=True).decode()
        
        return ""  # Return empty string if no content found

    def _extract_report_id(self, content: str) -> Optional[str]:
        """Extract expense report ID from email content."""
        # This is a simple example. Adjust extraction logic as needed.
        if "ID:" in content:
            return content.split("ID:")[1].split()[0]
        return None

    def _extract_decision(self, content: str) -> Optional[str]:
        """Extract approval/rejection decision from email content."""
        content = content.lower()
        if "approve" in content:
            return "approved"
        elif "reject" in content:
            return "rejected"
        return None

    def _log_decision(self, report_id: str, decision: str):
        """Log manager's decision."""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        
        # Save to log file (in the current directory)
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

# Instantiate the email service
email_service = EmailService()

# API Routes
@app.post("/send-summary-email")
async def send_summary_email(request: EmailRequest, background_tasks: BackgroundTasks):
    """API endpoint to send summary email."""
    background_tasks.add_task(email_service.send_summary_email, request)
    return {"status": "success", "message": "Email sending initiated"}

@app.post("/process-email-replies")
async def process_email_replies_endpoint():
    """API endpoint to process manager replies."""
    try:
        # Process emails synchronously before sending response
        result = await email_service.process_email_replies()
        return result
    except Exception as e:
        logger.error(f"Error in process_email_replies endpoint: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    """Handle approval via email link click."""
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    """Handle rejection via email link click."""
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

# Function to run uvicorn in a background thread
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the server in a background thread so the cell doesn't block
thread = threading.Thread(target=run_app, daemon=True)
thread.start()

print("FastAPI application is running at http://localhost:8000")


In [ ]:
from fastapi import FastAPI, BackgroundTasks, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import smtplib
import imaplib
import email
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import logging
from datetime import datetime
import json
from pathlib import Path

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load configuration from environment variables
GMAIL_USER = os.getenv("GMAIL_USER")
GMAIL_PASSWORD = os.getenv("GMAIL_APP_PASSWORD")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

app = FastAPI()

class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        
    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        """Generate HTML email content with expense reports and action buttons."""
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        
        for report in expense_reports:
            approve_url = f"{API_BASE_URL}/approve/{report.id}"
            reject_url = f"{API_BASE_URL}/reject/{report.id}"
            
            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """
            
        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        """Send summary email to manager with expense reports."""
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            # Create HTML content
            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            # Attach receipts if available
            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                        filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            # Send email
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        """Process incoming email replies for approvals/rejections."""
        try:
            # Connect to IMAP server
            mail = imaplib.IMAP4_SSL(self.imap_server)
            mail.login(GMAIL_USER, GMAIL_PASSWORD)
            mail.select('inbox')

            # Search for unread emails
            _, messages = mail.search(None, 'UNSEEN')
            
            for msg_num in messages[0].split():
                _, msg_data = mail.fetch(msg_num, '(RFC822)')
                email_body = msg_data[0][1]
                email_message = email.message_from_bytes(email_body)
                
                # Process email content
                subject = email_message['subject']
                if 'Re: Expense Reports Requiring Approval' in subject:
                    content = self._get_email_content(email_message)
                    
                    # Extract expense report ID and decision
                    report_id = self._extract_report_id(content)
                    decision = self._extract_decision(content)
                    
                    if report_id and decision:
                        self._log_decision(report_id, decision)
                        logger.info(f"Processed decision: {decision} for report {report_id}")

            return {"status": "success", "message": "Processed email replies"}

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    def _get_email_content(self, email_message) -> str:
        """Extract email content from message."""
        if email_message.is_multipart():
            for part in email_message.walk():
                if part.get_content_type() == "text/plain":
                    return part.get_payload(decode=True).decode()
        return email_message.get_payload(decode=True).decode()

    def _extract_report_id(self, content: str) -> Optional[str]:
        """Extract expense report ID from email content."""
        # Implementation would depend on email format
        # This is a simple example
        if "ID:" in content:
            return content.split("ID:")[1].split()[0]
        return None

    def _extract_decision(self, content: str) -> Optional[str]:
        """Extract approval/rejection decision from email content."""
        content = content.lower()
        if "approve" in content:
            return "approved"
        elif "reject" in content:
            return "rejected"
        return None

    def _log_decision(self, report_id: str, decision: str):
        """Log manager's decision."""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        
        # Save to log file
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

email_service = EmailService()

@app.post("/send-summary-email")
async def send_summary_email(request: EmailRequest, background_tasks: BackgroundTasks):
    """API endpoint to send summary email."""
    background_tasks.add_task(email_service.send_summary_email, request)
    return {"status": "success", "message": "Email sending initiated"}

@app.post("/process-email-replies")
async def process_email_replies(background_tasks: BackgroundTasks):
    """API endpoint to process manager replies."""
    background_tasks.add_task(email_service.process_email_replies)
    return {"status": "success", "message": "Email processing initiated"}

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    """Handle approval via email link click."""
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    """Handle rejection via email link click."""
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

In [ ]:
import os
import json
import logging
import smtplib
import imaplib
import email
from datetime import datetime
from pathlib import Path
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from typing import List, Optional

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# For running FastAPI in Jupyter Notebook
import nest_asyncio
import uvicorn

# Apply nest_asyncio
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
GMAIL_USER = os.getenv("GMAIL_USER", "xtractpay@gmail.com")
GMAIL_PASSWORD = os.getenv("GMAIL_APP_PASSWORD", "qzbqhlvmvjsdtana")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

# Data Models
class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

# Initialize FastAPI
app = FastAPI()

# Email Service
class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        
    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        for report in expense_reports:
            approve_url = f"{API_BASE_URL}/approve/{report.id}"
            reject_url = f"{API_BASE_URL}/reject/{report.id}"
            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """
        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                        filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        try:
            mail = imaplib.IMAP4_SSL(self.imap_server)
            mail.login(GMAIL_USER, GMAIL_PASSWORD)
            mail.select('inbox')

            _, messages = mail.search(None, 'UNSEEN')
            
            if not messages[0]:
                mail.close()
                mail.logout()
                return {"status": "success", "message": "No new emails found", "replies": []}
            
            replies = []
            for msg_num in messages[0].split():
                try:
                    _, msg_data = mail.fetch(msg_num, '(RFC822)')
                    if not msg_data or not msg_data[0]:
                        continue
                        
                    email_body = msg_data[0][1]
                    email_message = email.message_from_bytes(email_body)
                    
                    content = self._get_email_content(email_message)
                    
                    reply = {
                        "from": email_message['from'],
                        "subject": email_message['subject'],
                        "content": content,
                        "date": email_message['date']
                    }
                    
                    replies.append(reply)
                    print(json.dumps(reply, indent=2))
                    
                except Exception as e:
                    logger.error(f"Error processing individual email: {str(e)}")
                    continue
            
            mail.close()
            mail.logout()
            
            return {
                "status": "success", 
                "message": f"Processed {len(replies)} emails", 
                "replies": replies
            }

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    def _get_email_content(self, email_message) -> str:
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain":
                    return part.get_payload(decode=True).decode()
                elif content_type == "text/html":
                    return part.get_payload(decode=True).decode()
        else:
            return email_message.get_payload(decode=True).decode()
        return ""

    def _log_decision(self, report_id: str, decision: str):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

# Create email service instance
email_service = EmailService()

# API Routes
@app.post("/send-summary-email")
async def send_summary_email_endpoint(request: EmailRequest):
    return await email_service.send_summary_email(request)

@app.post("/process-email-replies")
async def process_email_replies_endpoint():
    return await email_service.process_email_replies()

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

# Run the server
if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

In [ ]:
import os
import json
import logging
import smtplib
import imaplib
import email
from datetime import datetime
from pathlib import Path
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from typing import List, Optional

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# For running FastAPI in Jupyter Notebook
import nest_asyncio
import uvicorn

# Apply nest_asyncio
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
GMAIL_USER = os.getenv("GMAIL_USER", "xtractpay@gmail.com")
GMAIL_PASSWORD = os.getenv("GMAIL_APP_PASSWORD", "qzbqhlvmvjsdtana")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

# Data Models
class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

# Initialize FastAPI
app = FastAPI()

# Email Service
class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        self.imap_port = 993  # Adding explicit IMAP port
        self.timeout = 30  # Adding timeout configuration

    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        for report in expense_reports:
            approve_url = f"{API_BASE_URL}/approve/{report.id}"
            reject_url = f"{API_BASE_URL}/reject/{report.id}"
            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """
        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                        filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        mail = None
        try:
            # Create IMAP connection with timeout
            mail = imaplib.IMAP4_SSL(self.imap_server, self.imap_port, timeout=self.timeout)
            
            # Login with error handling
            try:
                mail.login(GMAIL_USER, GMAIL_PASSWORD)
            except imaplib.IMAP4.error as e:
                logger.error(f"IMAP login failed: {str(e)}")
                raise HTTPException(status_code=401, detail="Email authentication failed")
            
            # Select inbox with error handling
            try:
                mail.select('inbox')
            except imaplib.IMAP4.error as e:
                logger.error(f"Failed to select inbox: {str(e)}")
                raise HTTPException(status_code=500, detail="Failed to access inbox")

            # Search for unseen messages
            try:
                _, messages = mail.search(None, 'UNSEEN')
            except imaplib.IMAP4.error as e:
                logger.error(f"Failed to search messages: {str(e)}")
                raise HTTPException(status_code=500, detail="Failed to search messages")
            
            if not messages[0]:
                return {"status": "success", "message": "No new emails found", "replies": []}
            
            replies = []
            for msg_num in messages[0].split():
                try:
                    _, msg_data = mail.fetch(msg_num, '(RFC822)')
                    if not msg_data or not msg_data[0]:
                        continue
                        
                    email_body = msg_data[0][1]
                    email_message = email.message_from_bytes(email_body)
                    
                    content = self._get_email_content(email_message)
                    
                    reply = {
                        "from": email_message['from'],
                        "subject": email_message['subject'],
                        "content": content,
                        "date": email_message['date']
                    }
                    
                    replies.append(reply)
                    logger.info(f"Processed email from: {reply['from']}")
                    
                except Exception as e:
                    logger.error(f"Error processing individual email: {str(e)}")
                    continue
            
            return {
                "status": "success", 
                "message": f"Processed {len(replies)} emails", 
                "replies": replies
            }

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))
            
        finally:
            # Ensure proper cleanup of IMAP connection
            if mail:
                try:
                    mail.close()
                    mail.logout()
                except:
                    pass

    def _get_email_content(self, email_message) -> str:
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain":
                    return part.get_payload(decode=True).decode()
                elif content_type == "text/html":
                    return part.get_payload(decode=True).decode()
        else:
            return email_message.get_payload(decode=True).decode()
        return ""

    def _log_decision(self, report_id: str, decision: str):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

# Create email service instance
email_service = EmailService()

# API Routes
@app.post("/send-summary-email")
async def send_summary_email_endpoint(request: EmailRequest):
    return await email_service.send_summary_email(request)

@app.post("/process-email-replies")
async def process_email_replies_endpoint():
    return await email_service.process_email_replies()

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

# Run the server
if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

In [ ]:
import os
import json
import logging
import smtplib
import imaplib
import email
from datetime import datetime
from pathlib import Path
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from typing import List, Optional

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# For running FastAPI in Jupyter Notebook
import nest_asyncio
import uvicorn

# Apply nest_asyncio
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
GMAIL_USER = os.getenv("GMAIL_USER", "xtractpay@gmail.com")
GMAIL_PASSWORD = os.getenv("GMAIL_APP_PASSWORD", "qzbqhlvmvjsdtana")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

# Data Models
class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

# Initialize FastAPI
app = FastAPI()

# Email Service
class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        self.imap_port = 993  # Adding explicit IMAP port
        self.timeout = 60  # Increasing timeout configuration

    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
            html_content = """
            <html>
            <head>
                <style>
                    .expense-table { border-collapse: collapse; width: 100%; }
                    .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                    .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                    .button { 
                        display: inline-block; 
                        padding: 10px 20px; 
                        margin: 5px; 
                        text-decoration: none; 
                        border-radius: 5px; 
                        color: white;
                        cursor: pointer;
                        border: none;
                    }
                    .approve { background-color: #4CAF50; }
                    .reject { background-color: #f44336; }
                    .status-message {
                        display: none;
                        margin-top: 5px;
                        padding: 5px;
                        border-radius: 3px;
                    }
                    .success { 
                        display: block;
                        background-color: #dff0d8;
                        color: #3c763d;
                    }
                    .error { 
                        display: block;
                        background-color: #f2dede;
                        color: #a94442;
                    }
                </style>
            </head>
            <body>
                <h2>Expense Reports Requiring Approval</h2>
                <table class="expense-table">
                    <tr>
                        <th>ID</th>
                        <th>Vendor</th>
                        <th>Amount</th>
                        <th>Policy Violation</th>
                        <th>Actions</th>
                    </tr>
            """
            for report in expense_reports:
                html_content += f"""
                    <tr id="row_{report.id}">
                        <td>{report.id}</td>
                        <td>{report.vendor_name}</td>
                        <td>${report.amount:.2f}</td>
                        <td>{report.policy_violation}</td>
                        <td>
                            <button onclick="handleDecision('{report.id}', 'approve')" class="button approve">Approve</button>
                            <button onclick="handleDecision('{report.id}', 'reject')" class="button reject">Reject</button>
                            <div id="status_{report.id}" class="status-message"></div>
                        </td>
                    </tr>
                """
            
            html_content += """
                </table>
                
                <script>
                async function handleDecision(reportId, action) {
                    const statusDiv = document.getElementById(`status_${reportId}`);
                    const buttons = document.querySelectorAll(`#row_${reportId} button`);
                    
                    // Disable buttons while processing
                    console.log(`Processing ${action} for report ${reportId}`);
                    buttons.forEach(btn => btn.disabled = true);
                    
                    try {
                        const response = await fetch(`${window.location.origin}/${action}/${reportId}`, {
                            method: 'GET',
                            headers: {
                                'Accept': 'application/json'
                            }
                        });
                        
                        const result = await response.json();
                        
                        if (response.ok) {
                            statusDiv.textContent = `✓ ${result.message}`;
                            statusDiv.className = 'status-message success';
                            
                            // Hide buttons after successful action
                            buttons.forEach(btn => btn.style.display = 'none');
                        } else {
                            throw new Error(result.detail || 'Failed to process request');
                        }
                    } catch (error) {
                        statusDiv.textContent = `✗ Error: ${error.message}`;
                        statusDiv.className = 'status-message error';
                        
                        // Re-enable buttons on error
                        buttons.forEach(btn => btn.disabled = false);
                    }
                }
                </script>
            </body>
            </html>
            """
            return html_content

    async def send_summary_email(self, request: EmailRequest):
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            for report in request.expense_reports:
                if report.receipt_path:
                    try:  # Added try-except for file handling
                        with open(report.receipt_path, 'rb') as f:
                            receipt = MIMEApplication(f.read(), _subtype='pdf')
                            receipt.add_header('Content-Disposition', 'attachment',
                                            filename=f'receipt_{report.id}.pdf')
                            msg.attach(receipt)
                    except FileNotFoundError:
                        logger.error(f"Receipt file not found: {report.receipt_path}")
                        continue # Skip to the next report
                    except Exception as e:
                        logger.error(f"Error attaching receipt {report.receipt_path}: {e}")
                        continue

            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}
        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        mail = None
        try:
            # Create IMAP connection with timeout
            logger.info("Attempting to connect to IMAP server...")
            mail = imaplib.IMAP4_SSL(self.imap_server, self.imap_port, timeout=self.timeout)

            # Login with error handling
            try:
                mail.login(GMAIL_USER, GMAIL_PASSWORD)
                logger.info("IMAP login successful.")
            except imaplib.IMAP4.error as e:
                logger.error(f"IMAP login failed: {str(e)}")
                raise HTTPException(status_code=401, detail="Email authentication failed")
            except Exception as e:
                logger.error(f"Unexpected error during IMAP login: {str(e)}")
                raise HTTPException(status_code=500, detail="Unexpected error during login")

            # Select inbox with error handling
            try:
                mail.select('inbox')
                logger.info("Inbox selected successfully.")
            except imaplib.IMAP4.error as e:
                logger.error(f"Failed to select inbox: {str(e)}")
                raise HTTPException(status_code=500, detail="Failed to access inbox")
            except Exception as e:
                logger.error(f"Unexpected error selecting inbox: {str(e)}")
                raise HTTPException(status_code=500, detail="Unexpected error selecting inbox")

            # Search for unseen messages
            try:
                _, messages = mail.search(None, 'UNSEEN')
                logger.info(f"Search for unseen messages returned: {messages}")
            except imaplib.IMAP4.error as e:
                logger.error(f"Failed to search messages: {str(e)}")
                raise HTTPException(status_code=500, detail="Failed to search messages")
            except Exception as e:
                logger.error(f"Unexpected error during message search: {str(e)}")
                raise HTTPException(status_code=500, detail="Unexpected error during message search")

            if not messages[0]:
                logger.info("No new emails found.")
                return {"status": "success", "message": "No new emails found", "replies": []}

            replies = []
            for msg_num in messages[0].split():
                try:
                    _, msg_data = mail.fetch(msg_num, '(RFC822)')
                    if not msg_data or not msg_data[0]:
                        logger.warning(f"No data found for message {msg_num}, skipping.")
                        continue

                    email_body = msg_data[0][1]
                    email_message = email.message_from_bytes(email_body)

                    content = self._get_email_content(email_message)

                    reply = {
                        "from": email_message['from'],
                        "subject": email_message['subject'],
                        "content": content,
                        "date": email_message['date']
                    }

                    replies.append(reply)
                    logger.info(f"Processed email from: {reply['from']}")

                except Exception as e:
                    logger.error(f"Error processing individual email {msg_num}: {str(e)}")
                    continue

            return {
                "status": "success",
                "message": f"Processed {len(replies)} emails",
                "replies": replies
            }

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

        finally:
            # Ensure proper cleanup of IMAP connection
            if mail:
                try:
                    mail.close()
                    mail.logout()
                    logger.info("IMAP connection closed and logged out.")
                except Exception as e:
                    logger.error(f"Error during IMAP logout/close: {str(e)}")

    def _get_email_content(self, email_message) -> str:
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain":
                    return part.get_payload(decode=True).decode()
                elif content_type == "text/html":
                    return part.get_payload(decode=True).decode()
        else:
            return email_message.get_payload(decode=True).decode()
        return ""

    def _log_decision(self, report_id: str, decision: str):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

# Create email service instance
email_service = EmailService()

# API Routes
@app.post("/send-summary-email")
async def send_summary_email_endpoint(request: EmailRequest):
    return await email_service.send_summary_email(request)

@app.post("/process-email-replies")
async def process_email_replies_endpoint():
    return await email_service.process_email_replies()

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

# Run the server
if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()


In [ ]:
import os
import json
import logging
import smtplib
import imaplib
import email
from datetime import datetime
from pathlib import Path
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from typing import List, Optional

from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

# For running FastAPI in Jupyter Notebook
import nest_asyncio
import uvicorn

# Apply nest_asyncio
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
GMAIL_USER = os.getenv("GMAIL_USER", "xtractpay@gmail.com")
GMAIL_PASSWORD = os.getenv("GMAIL_APP_PASSWORD", "qzbqhlvmvjsdtana")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

# Data Models
class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

# Initialize FastAPI
app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

# Email Service
class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        self.imap_port = 993
        self.timeout = 30

    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        html_content = """
        <html>
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <style>
                .expense-table { border-collapse: collapse; width: 100%; margin-bottom: 20px; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 12px; text-align: left; }
                .expense-table th { background-color: #f5f5f5; }
                .expense-table tr:nth-child(even) { background-color: #f9f9f9; }
                .button { 
                    display: inline-block; 
                    padding: 8px 16px; 
                    margin: 4px; 
                    border-radius: 4px; 
                    color: white;
                    cursor: pointer;
                    border: none;
                    font-size: 14px;
                    font-weight: 500;
                    transition: background-color 0.3s;
                }
                .button:disabled {
                    opacity: 0.6;
                    cursor: not-allowed;
                }
                .approve { background-color: #4CAF50; }
                .approve:hover:not(:disabled) { background-color: #45a049; }
                .reject { background-color: #f44336; }
                .reject:hover:not(:disabled) { background-color: #da190b; }
                .status-message {
                    margin-top: 8px;
                    padding: 8px;
                    border-radius: 4px;
                    font-size: 14px;
                    display: none;
                }
                .success { 
                    display: block;
                    background-color: #dff0d8;
                    color: #3c763d;
                    border: 1px solid #d6e9c6;
                }
                .error { 
                    display: block;
                    background-color: #f2dede;
                    color: #a94442;
                    border: 1px solid #ebccd1;
                }
            </style>
        </head>
        <body>
            <h2 style="color: #333; margin-bottom: 20px;">Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        
        for report in expense_reports:
            html_content += f"""
                <tr id="row_{report.id}">
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <div class="action-buttons">
                            <button onclick="handleAction('{report.id}', 'approve')" class="button approve">Approve</button>
                            <button onclick="handleAction('{report.id}', 'reject')" class="button reject">Reject</button>
                            <div id="status_{report.id}" class="status-message"></div>
                        </div>
                    </td>
                </tr>
            """

        html_content += """
            </table>
            
            <script>
            async function handleAction(reportId, action) {
                const statusDiv = document.getElementById(`status_${reportId}`);
                const row = document.getElementById(`row_${reportId}`);
                const buttons = row.querySelectorAll('button');
                
                // Disable all buttons in the row
                buttons.forEach(btn => btn.disabled = true);
                
                try {
                    const apiUrl = `http://localhost:8000/${action}/${reportId}`;
                    console.log('Making request to:', apiUrl);
                    
                    const response = await fetch(apiUrl, {
                        method: 'GET',
                        headers: {
                            'Accept': 'application/json',
                            'Content-Type': 'application/json'
                        }
                    });
                    
                    const result = await response.json();
                    console.log('Response:', result);
                    
                    if (response.ok) {
                        statusDiv.textContent = `✓ ${action.charAt(0).toUpperCase() + action.slice(1)}d successfully`;
                        statusDiv.className = 'status-message success';
                        
                        // Hide the buttons
                        buttons.forEach(btn => btn.style.display = 'none');
                        
                        // Add visual feedback
                        row.style.backgroundColor = action === 'approve' ? '#efffef' : '#fff0f0';
                    } else {
                        throw new Error(result.detail || `Failed to ${action}`);
                    }
                } catch (error) {
                    console.error('Error:', error);
                    statusDiv.textContent = `✗ Error: ${error.message}`;
                    statusDiv.className = 'status-message error';
                    
                    // Re-enable the buttons on error
                    buttons.forEach(btn => btn.disabled = false);
                }
            }
            </script>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                        filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        mail = None
        try:
            mail = imaplib.IMAP4_SSL(self.imap_server, self.imap_port, timeout=self.timeout)
            
            try:
                mail.login(GMAIL_USER, GMAIL_PASSWORD)
            except imaplib.IMAP4.error as e:
                logger.error(f"IMAP login failed: {str(e)}")
                raise HTTPException(status_code=401, detail="Email authentication failed")
            
            try:
                mail.select('inbox')
            except imaplib.IMAP4.error as e:
                logger.error(f"Failed to select inbox: {str(e)}")
                raise HTTPException(status_code=500, detail="Failed to access inbox")

            try:
                _, messages = mail.search(None, 'UNSEEN')
            except imaplib.IMAP4.error as e:
                logger.error(f"Failed to search messages: {str(e)}")
                raise HTTPException(status_code=500, detail="Failed to search messages")
            
            if not messages[0]:
                return {"status": "success", "message": "No new emails found", "replies": []}
            
            replies = []
            for msg_num in messages[0].split():
                try:
                    _, msg_data = mail.fetch(msg_num, '(RFC822)')
                    if not msg_data or not msg_data[0]:
                        continue
                    
                    email_body = msg_data[0][1]
                    email_message = email.message_from_bytes(email_body)
                    content = self._get_email_content(email_message)
                    
                    reply = {
                        "from": email_message['from'],
                        "subject": email_message['subject'],
                        "content": content,
                        "date": email_message['date']
                    }
                    
                    replies.append(reply)
                    logger.info(f"Processed email from: {reply['from']}")
                    
                except Exception as e:
                    logger.error(f"Error processing individual email: {str(e)}")
                    continue
            
            return {
                "status": "success", 
                "message": f"Processed {len(replies)} emails", 
                "replies": replies
            }

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))
            
        finally:
            if mail:
                try:
                    mail.close()
                    mail.logout()
                except:
                    pass

    def _get_email_content(self, email_message) -> str:
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain":
                    return part.get_payload(decode=True).decode()
                elif content_type == "text/html":
                    return part.get_payload(decode=True).decode()
        else:
            return email_message.get_payload(decode=True).decode()
        return ""

    def _log_decision(self, report_id: str, decision: str):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

# Create email service instance
email_service = EmailService()

# API Routes
@app.post("/send-summary-email")
async def send_summary_email_endpoint(request: EmailRequest):
    return await email_service.send_summary_email(request)

@app.post("/process-email-replies")
async def process_email_replies_endpoint():
    return await email_service.process_email_replies()

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str, request: Request):
    email_service._log_decision(report_id, "approved")
    client_host = request.client.host
    logger.info(f"Approval request from {client_host} for report {report_id}")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str, request: Request):
    email_service._log_decision(report_id, "rejected")
    client_host = request.client.host
    logger.info(f"Rejection request from {client_host} for report {report_id}")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

# Run the server
if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

In [ ]:
import os
import json
import logging
import smtplib
import imaplib
import email
from datetime import datetime
from pathlib import Path
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from typing import List, Optional

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# For running FastAPI in Jupyter Notebook
import nest_asyncio
import uvicorn

# Apply nest_asyncio
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
GMAIL_USER = os.getenv("GMAIL_USER", "xtractpay@gmail.com")
GMAIL_PASSWORD = os.getenv("GMAIL_APP_PASSWORD", "qzbqhlvmvjsdtana")
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")

# Data Models
class ExpenseReport(BaseModel):
    id: str
    vendor_name: str
    amount: float
    policy_violation: str
    receipt_path: Optional[str] = None

class EmailRequest(BaseModel):
    manager_email: str
    expense_reports: List[ExpenseReport]

# Initialize FastAPI
app = FastAPI()

# Email Service
class EmailService:
    def __init__(self):
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        self.imap_server = "imap.gmail.com"
        
    def create_html_content(self, expense_reports: List[ExpenseReport]) -> str:
        html_content = """
        <html>
        <head>
            <style>
                .expense-table { border-collapse: collapse; width: 100%; }
                .expense-table th, .expense-table td { border: 1px solid #ddd; padding: 8px; }
                .expense-table tr:nth-child(even) { background-color: #f2f2f2; }
                .button { display: inline-block; padding: 10px 20px; margin: 5px; 
                         text-decoration: none; border-radius: 5px; color: white; }
                .approve { background-color: #4CAF50; }
                .reject { background-color: #f44336; }
            </style>
        </head>
        <body>
            <h2>Expense Reports Requiring Approval</h2>
            <table class="expense-table">
                <tr>
                    <th>ID</th>
                    <th>Vendor</th>
                    <th>Amount</th>
                    <th>Policy Violation</th>
                    <th>Actions</th>
                </tr>
        """
        for report in expense_reports:
            approve_url = f"{API_BASE_URL}/approve/{report.id}"
            reject_url = f"{API_BASE_URL}/reject/{report.id}"
            html_content += f"""
                <tr>
                    <td>{report.id}</td>
                    <td>{report.vendor_name}</td>
                    <td>${report.amount:.2f}</td>
                    <td>{report.policy_violation}</td>
                    <td>
                        <a href="{approve_url}" class="button approve">Approve</a>
                        <a href="{reject_url}" class="button reject">Reject</a>
                    </td>
                </tr>
            """
        html_content += """
            </table>
        </body>
        </html>
        """
        return html_content

    async def send_summary_email(self, request: EmailRequest):
        try:
            msg = MIMEMultipart()
            msg['From'] = GMAIL_USER
            msg['To'] = request.manager_email
            msg['Subject'] = f"Expense Reports Requiring Approval - {datetime.now().strftime('%Y-%m-%d')}"

            html_content = self.create_html_content(request.expense_reports)
            msg.attach(MIMEText(html_content, 'html'))

            for report in request.expense_reports:
                if report.receipt_path:
                    with open(report.receipt_path, 'rb') as f:
                        receipt = MIMEApplication(f.read(), _subtype='pdf')
                        receipt.add_header('Content-Disposition', 'attachment', 
                                        filename=f'receipt_{report.id}.pdf')
                        msg.attach(receipt)

            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(GMAIL_USER, GMAIL_PASSWORD)
                server.send_message(msg)

            logger.info(f"Summary email sent to {request.manager_email}")
            return {"status": "success", "message": "Email sent successfully"}

        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    async def process_email_replies(self):
        try:
            mail = imaplib.IMAP4_SSL(self.imap_server)
            mail.login(GMAIL_USER, GMAIL_PASSWORD)
            mail.select('inbox')

            _, messages = mail.search(None, 'UNSEEN')
            
            if not messages[0]:
                mail.close()
                mail.logout()
                return {"status": "success", "message": "No new emails found", "replies": []}
            
            replies = []
            for msg_num in messages[0].split():
                try:
                    _, msg_data = mail.fetch(msg_num, '(RFC822)')
                    if not msg_data or not msg_data[0]:
                        continue
                        
                    email_body = msg_data[0][1]
                    email_message = email.message_from_bytes(email_body)
                    
                    content = self._get_email_content(email_message)
                    
                    reply = {
                        "from": email_message['from'],
                        "subject": email_message['subject'],
                        "content": content,
                        "date": email_message['date']
                    }
                    
                    replies.append(reply)
                    print(json.dumps(reply, indent=2))
                    
                except Exception as e:
                    logger.error(f"Error processing individual email: {str(e)}")
                    continue
            
            mail.close()
            mail.logout()
            
            return {
                "status": "success", 
                "message": f"Processed {len(replies)} emails", 
                "replies": replies
            }

        except Exception as e:
            logger.error(f"Error processing email replies: {str(e)}")
            raise HTTPException(status_code=500, detail=str(e))

    def _get_email_content(self, email_message) -> str:
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == "text/plain":
                    return part.get_payload(decode=True).decode()
                elif content_type == "text/html":
                    return part.get_payload(decode=True).decode()
        else:
            return email_message.get_payload(decode=True).decode()
        return ""

    def _log_decision(self, report_id: str, decision: str):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "report_id": report_id,
            "decision": decision
        }
        log_file = Path("decisions.log")
        with open(log_file, "a") as f:
            json.dump(log_entry, f)
            f.write("\n")

# Create email service instance
email_service = EmailService()

# API Routes
@app.post("/send-summary-email")
async def send_summary_email_endpoint(request: EmailRequest):
    return await email_service.send_summary_email(request)

@app.post("/process-email-replies")
async def process_email_replies_endpoint():
    return await email_service.process_email_replies()

@app.get("/approve/{report_id}")
async def approve_expense(report_id: str):
    email_service._log_decision(report_id, "approved")
    return {"status": "success", "message": f"Expense report {report_id} approved"}

@app.get("/reject/{report_id}")
async def reject_expense(report_id: str):
    email_service._log_decision(report_id, "rejected")
    return {"status": "success", "message": f"Expense report {report_id} rejected"}

# Run the server
if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

INFO:     Started server process [26136]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
2025-02-08 19:14:05,797 - __main__ - INFO - Summary email sent to bhavikpunmiya@gmail.com


INFO:     127.0.0.1:60723 - "POST /send-summary-email HTTP/1.1" 200 OK
